# Wykrywanie wad Ball Grid Array za pomocą ML

### Rozpoznanie defektu typu void dla układu "Ball Grid Array" przy pomocy machine learning.

IPC-A-610, zatytułowany „Akceptowalność zespołów elektronicznych”, jest powszechnie uznawanym standardem w branży elektronicznej, który zapewnia kompleksowe kryteria akceptacji zespołów elektronicznych, w tym tych wykorzystujących komponenty Ball Grid Array (BGA).

Jeśli chodzi o BGA, IPC-A-610 zajmuje się kilkoma krytycznymi aspektami, aby zapewnić jakość montażu:

Jakość połączenia lutowanego: Norma określa akceptowalne warunki dla połączeń lutowanych BGA, skupiając się na takich czynnikach, jak przesunięcie kulki lutowniczej, kształt i obecność pustych przestrzeni. Na przykład IPC-A-610F definiuje, że kulka BGA jest akceptowalna dla klas 1, 2 i 3, jeśli jej powierzchnia pusta jest mniejsza niż 30% powierzchni kuli w 2D kontroli rentgenowskiej.

Inspekcja i testowanie: Norma określa zalecane techniki inspekcji, takie jak analiza rentgenowska, w celu oceny integralności połączeń lutowanych BGA i wykrywania potencjalnych wad, takich jak pustki lub nieprawidłowe ustawienie.

Przestrzeganie kryteriów IPC-A-610 dla zespołów BGA pomaga zapewnić niezawodność i wydajność produktu, co jest szczególnie ważne w zastosowaniach wymagających wysokiej jakości zespołów elektronicznych.

PCB:
<img src="https://static.wixstatic.com/media/74cd3b_773e6cd81fb1433e972e27dbf33c53bd~mv2_d_4000_3000_s_4_2.jpg" alt="PCB" width="200">
- Źródło: [regulus-ems.com](https://www.regulus-ems.com/leaded-pcb-assembly)

BGA:
<img src="https://www.fs-pcba.com/wp-content/uploads/2022/12/1-9.jpg" alt="BGA" width="200">
- Źródło: [madpcb.com](https://madpcb.com/glossary/pbga/)


BGA solder ball:
<img src="https://epp-europe-news.com/wp-content/uploads/3/6/3603682.jpg" alt="Ball" width="200">

Źródło: [epp-europe-news.com](https://epp-europe-news.com/technology/applications/proper-inspection-strategy/)


X-Ray image:

| Przykład dobrych połączeń                                                                                                                      | Przykład wadliwych połączeń                                                                                                                      |
|------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------|
| <img src="https://github.com/Mat3u52/BallGridArrayDefectDetectionByML/blob/main/examples/good_image.jpg?raw=true" alt="Good BGA" width="200">  | <img src="https://github.com/Mat3u52/BallGridArrayDefectDetectionByML/blob/main/examples/not_good_image.jpg?raw=true" alt="Bad BGA" width="200"> |
| <img src="https://github.com/Mat3u52/BallGridArrayDefectDetectionByML/blob/main/examples/single_good_ball.png?raw=true" alt="Good BGA" width="200">  | <img src="https://github.com/Mat3u52/BallGridArrayDefectDetectionByML/blob/main/examples/single_not_good_ball.png?raw=true" alt="Good BGA" width="200"> |




## 1. Przygotowanie zestawu danych

### Import potrzebnych bibliotek

In [ ]:
import os
import sys
import random
import cv2
import shutil
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from zipfile import ZipFile
from typing import List
from PIL import Image

### Informacje o pliku „Dataset.zip”
Przygotowana paczka zawiera zestaw plików w różnych formatach, w tym obrazy ATG układu BGA po montażu zapisane w formacie .png.
.
      



In [ ]:
def print_zip_summary(zip_path: str) -> None:
    """
    Prints a summary of the contents of a ZIP file.

    Given the path to a ZIP file, this function opens the file in read-only mode,
    retrieves a list of all the files contained within it, and prints a summary of the ZIP file.
    Specifically, it displays:
    - The base name of the ZIP file.
    - The file size of the ZIP archive in bytes.
    - The total number of files contained within the archive.
    - A list of the last 5 files in the archive, if the archive contains that many.
      If the ZIP archive has fewer than 5 files, it will print all of them.

    :param zip_path: str, the path to the ZIP file to summarize.
    :return: None
    """
    with ZipFile(zip_path, 'r') as zip_file:
        files: List[str] = zip_file.namelist()

        print(f"File name: {os.path.basename(zip_path)}")
        print(f"File size: {os.path.getsize(zip_path)}")
        print(f"Total files in ZIP: {len(files)}")
        print("Last 5 files in ZIP:")

        for file_name in files[-5:]:
            print(file_name)

  - Metoda "print_zip_summary":
  
          Podsumowanie zawartości pliku ZIP.
          Po podaniu ścieżki do pliku ZIP ta funkcja otwiera plik w trybie tylko do odczytu,
          pobiera listę wszystkich plików w nim zawartych i wyświetla podsumowanie pliku ZIP.
          
          W szczególności wyświetla:
            Nazwę bazową pliku ZIP. 
            Rozmiar pliku archiwum ZIP w bajtach.
            Całkowitą liczbę plików zawartych w archiwum.
            Listę ostatnich 5 plików w archiwum. Jeśli archiwum ZIP zawiera mniej niż 5 plików, wydrukuje je wszystkie.
      

In [ ]:
# general information of .zip file
print_zip_summary('Dataset.zip')

### Wypakowanie zawartości pliku ZIP

In [ ]:
def unzip_precess(zip_path: str, zip_path_dist: str) -> None:
    """
    Extracts files from a zip archive to a specified directory, showing extraction progress.

    :param zip_path: str, The path to the zip file to extract.
    :param zip_path_dist: str, The directory path where the contents of the zip file will be extracted.
    :return: None
    """
    if os.path.isfile(zip_path) and not os.path.isdir(zip_path_dist):
        with ZipFile(zip_path) as zip_file:
            files_list: List[str] = zip_file.namelist()
            for idx, file in enumerate(files_list):
                percent = round((idx / len(files_list)) * 100)
                sys.stdout.write(f"\runzip process: {percent}%")
                sys.stdout.flush()
                zip_file.extract(file, zip_path_dist)
            zip_file.close()
            sys.stdout.write("\runzip process: 100%\n")
            sys.stdout.flush()


  - Metoda "unzip_precess":
        
        Wypakowuje pliki z archiwum ZIP do określonego katalogu, pokazując postęp wypakowywania.
  

*Przed wywołaniem metody „unzip_process” w Jupyter Notebook warto zmodyfikować plik „jupyter_notebook_config” i dodać linię „c.ServerApp.iopub_msg_rate_limit = 50000”, w przeciwnym razie notebook zwróci komunikat „IOPub message rate exceeded…”.*

In [ ]:
    # unzipping
    unzip_precess('Dataset.zip', 'UnzippedDataset')

### Sprawdzenie zawartości katalogu "UnzippedDataset"

In [ ]:
def tree(directory: str, file_range: int = -5, indent: int = 0) -> None:
    """
    Recursively lists the contents of a directory in a structured, hierarchical format.

    :param directory: str, path to directory.
    :param file_range: int, amount of samples.
    :param indent: int, space iterator.
    :return: None.
    """
    try:
        entries: list[str] = os.listdir(directory)

        dir_entries: list[str] = [e for e in entries if os.path.isdir(os.path.join(directory, e))]
        file_entries: list[str] = [e for e in entries if os.path.isfile(os.path.join(directory, e))][file_range:]

        for d in dir_entries:
            print("    " * indent + f"[Directory] {d}/")
            tree(os.path.join(directory, d), file_range, indent + 1)

        for f in file_entries:
            if f[-4:] != '.png':
                print("    " * indent + f"[other file] - {f} [note]The file should be removed {os.path.join(directory, f)}")
            else:
                print("    " * indent + f"[png file] - {f}")

    except FileNotFoundError:
        print(f"Error: Directory '{directory}' not found.")
    except PermissionError:
        print(f"Error: Permission denied for directory '{directory}'.")

  - Metoda "tree":
  
        Rekurencyjnie wyświetla zawartość katalogu w ustrukturyzowanym, hierarchicznym formacie.

In [ ]:
    # directory and file tree
    tree('UnzippedDataset')

### Zaprezentowanie przykładowej próbki nieuporządkowanych danych.

In [ ]:
def show_samples(image_directory: str) -> None:
    """
    Randomly selects and displays ten .png images from a given directory.
    Each image is resized to 300x300 pixels and shown in a 2-row, 5-column plot.

    :param image_directory: str, The path to the directory containing image files.
    :return: None
    """
    all_files: list[str] = os.listdir(image_directory)
    image_files: list[str] = [f for f in all_files if f.endswith('.png')]

    if len(image_files) < 10:
        print("Not enough images in the directory to display 10 samples.")
        return
    
    random_images: list[str] = random.sample(image_files, 10)

    fig, axes = plt.subplots(2, 5, figsize=(15, 6))

    axes = axes.ravel()

    for ax, image_file in zip(axes, random_images):
        image_path: str = os.path.join(image_directory, image_file)
        img: np.ndarray = cv2.imread(image_path)

        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (300, 300))
            ax.imshow(img)
            ax.set_title(image_file, fontsize=10)
            ax.axis('off')

    plt.tight_layout()
    plt.show()

  - Metoda "show_samples":
  
        Losowo wybiera i wyświetla dziesięć obrazów .png z danego katalogu. Każdy obraz jest zmieniany na
        300x300 pikseli i wyświetlany w matplotlib. Na ten moment zestaw danych jest bardzo "zanieczyszczony" i nadmiarowy.


In [ ]:
    # Show samples
    show_samples('UnzippedDataset/Dataset/inspection/2024-10-31_15-09-15-335')

### Uporządkowanie danych

In [ ]:
def recognizer(image: str, image_id: int, margin: int = 7) -> None:
    """
    Processes an image to identify the largest circular object, calculates its diameter and area,
    and highlights void areas within a margin around the detected contour.

    :param image: str, Path to the input image file.
    :param image_id: int, Identifier for naming output files.
    :param margin: int, Margin size for detection (default is 7 pixels).
    """
    threshold = 110
    image_path = image

    img = Image.open(image)

    if img.mode != 'RGB':
        img = img.convert('RGB')

    pixels = list(img.getdata())
    new_pixels = [(0, 0, 0) if (p[0] if isinstance(p, tuple) else p) <= threshold else (255, 255, 255) for p in pixels]
    new_img = Image.new('RGB', img.size)
    new_img.putdata(new_pixels)
    new_img.save(f'labeled/bitmapDiagnostics/{image_id}_bitmap.jpg')

    binary_image = cv2.imread(f'labeled/bitmapDiagnostics/{image_id}_bitmap.jpg', cv2.IMREAD_GRAYSCALE)

    edges = cv2.Canny(binary_image, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    image_center = (binary_image.shape[1] // 2, binary_image.shape[0] // 2)
    best_contour = min(
        contours, key=lambda contour: cv2.pointPolygonTest(contour, image_center, True) ** 2, default=None
    )

    mask = np.zeros_like(binary_image)
    diameter = 0

    if best_contour is not None:
        ((x, y), radius) = cv2.minEnclosingCircle(best_contour)
        diameter = 2 * radius

        cv2.circle(mask, (int(x), int(y)), int(radius), 255, thickness=-1)

        kernel = np.ones((margin, margin), np.uint8)
        mask_with_margin = cv2.erode(mask, kernel, iterations=1)

        output_image = cv2.cvtColor(binary_image, cv2.COLOR_GRAY2BGR)
        cv2.drawContours(output_image, [best_contour], -1, (255, 0, 0), 2)

    else:
        output_image = cv2.cvtColor(binary_image, cv2.COLOR_GRAY2BGR)

    white_pixel_mask = (binary_image >= 200) & (mask_with_margin == 255) if best_contour is not None else None
    white_pixel_count = np.count_nonzero(white_pixel_mask) if white_pixel_mask is not None else 0

    if white_pixel_mask is not None:
        output_image[white_pixel_mask] = [0, 255, 255]

    plt.figure(figsize=(2, 2))
    plt.imshow(cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))
    plt.title(f'Detected Object with Diameter: {diameter:.2f} pixels')
    plt.axis('off')
    mask_plot_path = f"labeled/bitmapDiagnostics/{image_id}_edge_plot.png"
    plt.savefig(mask_plot_path, bbox_inches='tight', dpi=300)
    #plt.show()
    plt.close('all')

    plt.figure(figsize=(2, 2))
    plt.imshow(cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))
    plt.title(f'Detected Object with Diameter: {diameter:.2f} pixels')
    plt.axis('off')
    plt.text(
        10, 30, f'Void Area: {white_pixel_count} px', color='red', fontsize=14,
        bbox=dict(facecolor='white', alpha=0.8)
    )
    void_plot_path = f"labeled/bitmapDiagnostics/{image_id}_void_plot.png"
    plt.savefig(void_plot_path, bbox_inches='tight', dpi=300)
    plt.show()
    plt.close('all')

    circle_area = np.pi * (radius ** 2) if best_contour is not None else 0
    print(f'Diameter: {diameter:.2f} px')
    print(f'Circle Area: {circle_area:.2f} px²')
    print(f'Void Area (White Pixels): {white_pixel_count} px²')

    status: bool = False
    if white_pixel_count > 15:
        shutil.copyfile(image_path, f'labeled/fail/{image_id}_ball.png')
        print("Directory: Fail")
        status = False
    else:
        shutil.copyfile(image_path, f'labeled/pass/{image_id}_ball.png')
        print("Directory: Pass")
        status = True

    with open("SolderBallsSize.csv", mode="a", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow([diameter, circle_area, white_pixel_count, status])


  - Metoda "recognizer":

        Przetwarza obraz w celu zidentyfikowania największego okrągłego obiektu, 
        oblicza jego średnicę i powierzchnię oraz podświetla puste obszary w obrębie marginesu wokół wykrytego konturu.
        Funkcja zostaje wywołana w pętli 'for' w celu odseparowanie zdjęć z defektem i bez defektu. 
        Dodatkowo w katalogu 'bitmapDiagnostics' umieszczone zostają pliki diagnostyczne.


In [ ]:

    i: int = 0
    for root, _, files in os.walk('UnzippedDataset\\Dataset\\inspection'):
        for file in files:
            if file.endswith('_48_48_4.png'):
                print(f"Path: {os.path.join(root, file)}")
                print(f"Index: {i}")
                recognizer(os.path.join(root, file), i)
            i += 1

In [ ]:
# Show samples
print("\nPass directory:")
show_samples('labeled/pass')
print("\nFail directory:")
show_samples('labeled/fail')
    

### Analiza danych

In [ ]:
column_names = ["BallDiameter [px]", "BallArea [px]", "VoidArea [px]", "Status [bool]"]
df = pd.read_csv("SolderBallsSize.csv", header=None, names=column_names, encoding="utf-8")
df.tail()

Wydzielamy pierwsze 1000 etykiet klasy odpowiadających statusom True oraz False i przekształcamy je w dwie kategorie symbolizowane liczbami całkowitymi: 1 (True) i -1 (False), które przypisujemy do wektora y.

Następnie, ze zbioru 1000 przykładów uczących wydzielamy drugą kolumnę cech (BallArea) oraz trzecią kolumnę (VoidArea), a uzyskane wartości przypisujemy do macierzy cech X. Tak skonstruowane dane możemy zwizualizować jako dwuwymiarowy wykres punktowy.

In [ ]:
df_sample = df.iloc[:1000]

X = df_sample[["BallArea [px]", "VoidArea [px]"]].values
y = np.where(df_sample["Status [bool]"] == True, 1, -1)

plt.figure(figsize=(8, 6))
plt.scatter(X[y == 1, 0], X[y == 1, 1], color='red', marker='o', label='True')
plt.scatter(X[y == -1, 0], X[y == -1, 1], color='blue', marker='x', label='False')

plt.xlabel("BallArea [px]")
plt.ylabel("VoidArea [px]")
plt.legend(loc="upper left")
plt.title("Scatter plot of BallArea vs. VoidArea")

plt.show()

### Klasa "Perceptron" Rosenblatta

In [ ]:
class Perceptron(object):
    """
    Perceptron classifier.
    
    :param eta: float, Learning rate (between 0.0 and 1.0)
    :param n_iter: int, Passes over the training dataset.
    :param random_state: int, Random number generator seed for random weight initialization.

    :attr w_: 1d-array, Weights after fitting.
    :attr errors_: list, Number of misclassifications (updates) in each epoch.

    """
    def __init__(self, eta=0.01, n_iter=50, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state

    def fit(self, X, y):
        """
        Fit training data.

        :param X: {array-like}, shape = [n_examples, n_features] Training vectors, where n_examples is the number of examples and n_features is the number of features.
        :param y: array-like, shape = [n_examples] Target values.

        :return self: object

        """
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
        self.errors_ = []

        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X, y):
                update = self.eta * (target - self.predict(xi))
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self

    def net_input(self, X):
        """Calculate net input"""
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def predict(self, X):
        """Return class label after unit step"""
        return np.where(self.net_input(X) >= 0.0, 1, -1)

Pierwotna reguła uczenia perceptronu, opracowana przez Rosenblatta, przedstawia się następująco i można ją opisać w kilku etapach:

1. Ustaw wagi na 0 lub niewielkie, losowe wartości.
2. Dla każdego przykładu uczącego 𝑥:
     - Oblicz wartość wyjściową y.
     - Zaktualizuj wagi.

<img src="https://github.com/Mat3u52/BallGridArrayDefectDetectionByML/blob/main/examples/perceptron.jpg?raw=true" alt="perceptron" width="800" >

In [ ]:
ppn = Perceptron(eta=0.1, n_iter=10)
ppn.fit(X,y)
plt.plot(range(1, len(ppn.errors_)+1), ppn.errors_, marker='o')
plt.xlabel('Epoki')
plt.ylabel('Liczba aktualizacji')
plt.show()

Zmniejszenie liczby aktualizacji: Na początku liczba aktualizacji jest wysoka (ponad 30), ale szybko maleje w kolejnych epokach. Oznacza to, że model stopniowo uczy się lepiej klasyfikować dane, co zmniejsza potrzebę aktualizacji wag.

Stabilizacja: Po kilku epokach (około 7-10) liczba aktualizacji osiąga zero. To wskazuje, że model nauczył się poprawnie klasyfikować wszystkie próbki w zbiorze treningowym (dla danych liniowo separowalnych).

Efektywność uczenia: Szybkie zmniejszenie liczby błędów na początku oznacza, że przyjęta wartość współczynnika uczenia (eta=0.1) oraz liczba epok (n_iter=10) są odpowiednie dla tego problemu.

Dane liniowo separowalne: Ponieważ liczba błędów osiąga zero, można przypuszczać, że zbiór danych jest liniowo separowalny.

In [ ]:
def plot_decision_regions(X, y, classifier, resolution=0.02):

    # setup marker generator and color map
    markers = ('s', 'o', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.3, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    # plot class examples
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], 
                    y=X[y == cl, 1],
                    alpha=0.8, 
                    c=colors[idx],
                    marker=markers[idx], 
                    label=cl) 
                    #edgecolor='black')

Najpierw definiujemy liczbę barw (colors) i znaczników (markers), a następnie tworzymy mapę kolorów z listy barw za pomocą klasy ListedColormap. Następnie określamy minimalne i maksymalne wartości dwóch cech, które wykorzystujemy do wygenerowania siatki współrzędnych, tworząc tablice xx1 i xx2 za pomocą funkcji meshgrid.

Ponieważ klasyfikator został wytrenowany na dwóch wymiarach cech, konieczna jest modyfikacja tablic xx1 i xx2 oraz utworzenie macierzy o takiej samej liczbie kolumn, jak zbiór uczący. Dzięki temu możemy zastosować metodę predict do przewidywania etykiet klas dla poszczególnych elementów siatki.

### Wykres regionów decyzyjnych

In [ ]:
plot_decision_regions(X, y, classifier=ppn)
plt.xlabel("BallArea [px]")
plt.ylabel("VoidArea [px]")
plt.legend(loc='upper left')
plt.show()